### **2.2 SQL Analysis**

---

#### **Objectives**

Demonstrate advanced SQL skills by creating a database-driven analysis workflow and answering complex business questions through structured queries.

---

In [1]:
import psycopg2
import polars as pl
import pandas as pd
from tqdm import tqdm
from typing import List
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from psycopg2.extras import execute_values

In [9]:
class DataSqlIntegration(ABC):
      """
      Abstract base class for bank marketing data SQL integration operations.

      This class defines the interface for loading bank marketing data into a SQL database
      and querying results. Concrete implementations must provide the actual database
      interaction logic.

      Attributes:
            dataPath (str): Path to the data file to be processed
      """
      def __init__(self, dataPath: str) -> None:
            """
            Initialize the data integration handler.
            
            Args:
            dataPath: Path to the bank marketing data file
            """
            self.dataPath = dataPath

      @abstractmethod
      def createTable(self, connection: psycopg2.connect) -> None:
            """
            Create the bank_marketing table in the database.
            
            Args:
            connection: Active PostgreSQL database connection
            
            Raises:
            psycopg2.Error: If table creation fails
            """
            pass

      @abstractmethod
      def answerQueryAnswer(self, connection:psycopg2.connect, queryString: str, index: List[str]) -> pd.DataFrame:
            """
            Convert SQL query results into a pandas DataFrame.
            
            Args:
            queryRows: List of tuples from SQL query results
            index: List of index values for the DataFrame
            
            Returns:
            pd.DataFrame: Formatted DataFrame from query results
            """
            pass

      @abstractmethod
      def insertRecordsIntoTable(self, connection: psycopg2.connect, alreadyAdded: bool, batch_size: int = 100_000) -> None:
            """
            Insert records from the data file into the database table.

            Args:
            connection: Active PostgreSQL database connection

            Raises:
            psycopg2.Error: If data insertion fails
            """
            pass


class EcommerceDataSqlIntegration(DataSqlIntegration):
      """
      Concrete implementation of bank marketing data SQL integration.
      
      Provides PostgreSQL-specific implementation for loading bank marketing campaign data
      and converting query results to pandas DataFrames.
      """
      
      def createTable(self, connection: psycopg2.connect) -> None:
            """
            Create the ecommerce_events table with proper schema if it doesn't exist.
            """
            cursor = connection.cursor()
            try:
                  cursor.execute("""
                        CREATE TABLE IF NOT EXISTS ecommerce_events (
                        event_id BIGSERIAL PRIMARY KEY,
                        event_time TIMESTAMPTZ NOT NULL,
                        event_type VARCHAR(20) NOT NULL CHECK (event_type IN ('view', 'cart', 'purchase')),
                        product_id BIGINT,
                        category_id BIGINT,
                        brand TEXT,
                        price NUMERIC(12,2),
                        user_id BIGINT,
                        user_session TEXT,

                        isFreeItem BOOLEAN,
                        isLuxuryItem BOOLEAN,
                        isExtremeOutlier BOOLEAN,
                        isAbnormal BOOLEAN,
                        has_purchase BOOLEAN,
                        isMultiDaySession BOOLEAN,
                        hasCategoryCode BOOLEAN,

                        category_l1 TEXT,
                        category_l2 TEXT,
                        category_l3 TEXT,
                        isLowFrequencyCategory BOOLEAN,

                        sessionDuration REAL,
                        sessionStartHour SMALLINT,
                        sessionDayOfWeek TEXT,
                        isWeekendSession BOOLEAN,
                        isMidnightActivity BOOLEAN,
                        isDirectPurchase BOOLEAN,
                        isDirectPurchase_right BOOLEAN,
                        isAbandonedCart BOOLEAN,
                        advancedToCart BOOLEAN,
                        advancedToPurchase BOOLEAN,
                        EngagementDepth INTEGER,

                        popularityScore REAL,
                        categoryConversionRate REAL,
                        isInMultiCategories BOOLEAN
                        );
                  """)
                  connection.commit()
            except psycopg2.Error as e:
                  connection.rollback()
                  raise e

      def addTableIndexes(self, connection: psycopg2.connect) -> None:
            try:
                  cursor = connection.cursor()
                  cursor.execute("CREATE INDEX IF NOT EXISTS idx_user_id ON ecommerce_events(user_id);")
                  cursor.execute("CREATE INDEX IF NOT EXISTS idx_product_id ON ecommerce_events(product_id);")
                  cursor.execute("CREATE INDEX IF NOT EXISTS idx_event_type ON ecommerce_events(event_type);")
                  cursor.execute("CREATE INDEX IF NOT EXISTS idx_event_time_brin ON ecommerce_events USING BRIN(event_time);")
                  cursor.execute("CREATE INDEX IF NOT EXISTS idx_user_event_time ON ecommerce_events(user_id, event_time);")
                  cursor.execute("CREATE INDEX IF NOT EXISTS idx_product_event_type ON ecommerce_events(product_id, event_type);")
                  cursor.execute("CREATE INDEX IF NOT EXISTS idx_purchase_only ON ecommerce_events(product_id, event_time) WHERE event_type = 'purchase';")
                  cursor.execute("ANALYZE ecommerce_events;")
                  connection.commit()

            except psycopg2.Error as e:
                  connection.rollback()
            
            finally:
                  cursor.close()

      def insertRecordsIntoTable(self, connection: psycopg2.connect, alreadyAdded: bool, batchSize: int = 100_000):
            if alreadyAdded:
                  return

            try:
                  cursor = connection.cursor()

                  # Lazy read Parquet
                  lazy_df = pl.scan_parquet(self.dataPath)
                  df = lazy_df.collect()
                  total_rows = df.height

                  # Iterate in chunks with progress bar
                  for start in tqdm(range(0, total_rows, batchSize), desc="Inserting rows"):
                        end = min(start + batchSize, total_rows)
                        chunk = df[start:end]
                        data_tuples = [tuple(row) for row in chunk.to_numpy()]

                        if data_tuples:
                              execute_values(cursor, """
                                    INSERT INTO ecommerce_events (
                                          event_time, event_type, product_id, category_id, brand, price, user_id, user_session,
                                          isFreeItem, isLuxuryItem, isExtremeOutlier, isAbnormal, has_purchase, isMultiDaySession,
                                          hasCategoryCode,
                                          category_l1, category_l2, category_l3,
                                          isLowFrequencyCategory,
                                          sessionDuration, sessionStartHour, sessionDayOfWeek, isWeekendSession,
                                          isMidnightActivity, isDirectPurchase, isDirectPurchase_right,
                                          isAbandonedCart, advancedToCart, advancedToPurchase, EngagementDepth,
                                          popularityScore, categoryConversionRate,
                                          isInMultiCategories
                                    ) VALUES %s
                              """, data_tuples)
                              connection.commit()

            except Exception as e:
                  connection.rollback()
                  raise e

      def addSummaryTables(self, connection: psycopg2.connect) -> None:
            try:
                  cursor = connection.cursor()
                  cursor.execute("""
                        CREATE MATERIALIZED VIEW IF NOT EXISTS daily_revenue_summary AS
                        SELECT
                        DATE(event_time) AS event_date,
                        SUM(price) AS total_revenue,
                        COUNT(*) FILTER (WHERE event_type = 'purchase') AS total_purchases
                        FROM ecommerce_events
                        WHERE event_type = 'purchase'
                        GROUP BY DATE(event_time)
                        ORDER BY event_date;
                  """)
                  cursor.execute("""
                        CREATE MATERIALIZED VIEW IF NOT EXISTS product_perf_summary AS
                        SELECT
                        product_id,
                        COUNT(*) FILTER (WHERE event_type = 'view') AS total_views,
                        COUNT(*) FILTER (WHERE event_type = 'cart') AS total_cart,
                        COUNT(*) FILTER (WHERE event_type = 'purchase') AS total_purchases,
                        (COUNT(*) FILTER (WHERE event_type = 'purchase')::decimal 
                        / NULLIF(COUNT(*) FILTER (WHERE event_type = 'view'), 0)) AS conversion_rate
                        FROM ecommerce_events
                        GROUP BY product_id
                        ORDER BY conversion_rate DESC;
                  """)
                  cursor.execute("""
                        CREATE MATERIALIZED VIEW IF NOT EXISTS customer_rfm AS
                        SELECT
                        user_id,
                        DATE_PART('day', CURRENT_DATE - MAX(event_time) FILTER (WHERE event_type = 'purchase')) AS recency,
                        COUNT(*) FILTER (WHERE event_type = 'purchase') AS frequency,
                        COALESCE(SUM(price) FILTER (WHERE event_type = 'purchase'), 0) AS monetary
                        FROM
                        ecommerce_events
                        GROUP BY
                        user_id
                        ORDER BY
                        monetary DESC;
                  """)
                  connection.commit()

            except psycopg2.Error as e:
                  connection.rollback()
            
            finally:
                  cursor.close() 

      def answerQueryAnswer(self, connection: psycopg2.connect, queryString: str, index: List[str]) -> pd.DataFrame:
            cursor = connection.cursor()
            try:
                  cursor.execute(queryString)
                  queryRows = cursor.fetchall()
                  
                  # Debug: Print raw query results
                  print(f"Query returned {len(queryRows)} rows")
                  if queryRows:
                        print("First row sample:", queryRows[0])
                  
                  # Create DataFrame with proper column handling
                  if not queryRows:
                        return pd.DataFrame(index=index)
                        
                  # Convert to DataFrame with column names
                  df = pd.DataFrame.from_records(
                        queryRows,
                        columns=[desc[0] for desc in cursor.description],
                        index=index[:len(queryRows)]  # Ensure index matches row count
                  )
                  return df
                  
            except psycopg2.Error as e:
                  print(f"Database error: {e}")
                  return pd.DataFrame()
            finally:
                  cursor.close()

In [11]:
dataSqlIntegration = EcommerceDataSqlIntegration(dataPath="D:\programming\Data Analysis\E-commerceCustomerBehaviorAnalysis\Data\Processed\prsc_fe9_nov_2019.parquet")
connection = psycopg2.connect(database="ecommerceCustomers",host="localhost",user="postgres",
                                    password="postgres",port="5432")

In [ ]:
dataSqlIntegration.createTable(connection=connection)
dataSqlIntegration.insertRecordsIntoTable(connection=connection, alreadyAdded=False, batchSize=100000)
dataSqlIntegration.addTableIndexes(connection=connection)
dataSqlIntegration.addSummaryTables(connection=connection)


---

# **Task 2.2.1 – Database Design and Data Loading**

---

### 1) Database Schema Design

**Q1. What is the optimal table structure to normalize the e-commerce data while maintaining query performance?**
**A (Strategy):**

* Use a **single wide `ecommerce_events` table** (as you already did) because:

  * Data is clean → no need for staging tables.
  * Analysis and BI dashboards benefit from one denormalized table.
* Keep dimensions (`products`, `customers`, `categories`) optional for future expansion, but not required now.
* Code strategy: rely on your `createTable()` implementation that defines the schema with correct data types and constraints.

---

**Q2. How should we design indexes on user\_id, product\_id, and event\_time to optimize common analytical queries?**
**A (Strategy):**

* Create **BTREE indexes** on:

  * `user_id` → for customer-level queries.
  * `product_id` → for product performance analysis.
  * `event_type` → for funnel/conversion queries.
* Create **BRIN index** on `event_time` → for fast filtering by time ranges (cheap and scalable for 285M rows).
* Optionally, add composite `(user_id, event_time)` for recency analysis.
* Code strategy: extend `createTable()` to run `CREATE INDEX` after table creation.

---

**Q3. What foreign key relationships should be established between customers, products, and events tables?**
**A (Strategy):**

* Since all analysis happens in the **single wide table**, foreign keys aren’t required for integrity (data is already clean).
* If needed in the future:

  * `events.user_id → customers.user_id`
  * `events.product_id → products.product_id`
* Code strategy: skip FK constraints for now to keep inserts fast; revisit if you add dimension tables later.

---

### 2) Data Import and Validation

**Q4. How can we efficiently load 285 million records into a relational database with proper data type conversions?**
**A (Strategy):**

* You already stream from parquet → Polars → numpy tuples → Postgres.
* Optimize by:

  * Using `execute_values()` in batches (you already do this).
  * Tuning `batchSize` (100k is good; test 250k–500k for speed).
  * Running with autocommit off + commit per batch (as you implemented).
* Code strategy: keep your current `insertRecordsIntoTable()` logic.

---

**Q5. What SQL constraints should be implemented to ensure data quality during the import process?**
**A (Strategy):**

* Since data is **already clean**, keep constraints minimal for performance:

  * `NOT NULL` only where logically required (`event_time`, `event_type`).
  * `CHECK` on `event_type IN ('view','cart','purchase')`.
* Skip extra checks like duplicates or null validation (handled upstream in data pipeline).
* Code strategy: your current `createTable()` is sufficient.

---

**Q6. How do we handle duplicate records and maintain referential integrity during bulk data loading?**
**A (Strategy):**

* No duplicates exist → skip deduplication logic.
* Referential integrity is not enforced (since no FK constraints are required).
* Code strategy: your `insertRecordsIntoTable()` just streams parquet → Postgres directly, which is optimal.

---

### 3) Performance Optimization

**Q7. What partitioning strategy should be used for the events table based on event\_time to improve query performance?**
**A (Strategy):**

* Use **range partitioning by month** on `event_time` (optional future step).
* For now, with clean wide data, a **BRIN index** on `event_time` is likely enough.
* Code strategy:

  * Add BRIN index after table creation.
  * If queries slow down, evolve to monthly partitions.

---

**Q8. How should we implement proper indexing for both transactional and analytical workloads?**
**A (Strategy):**

* Transactional (row lookups): `BTREE` on PK (`event_id`).
* Analytical: `BRIN` on `event_time`, `BTREE` on `user_id`, `product_id`, `event_type`.
* Optional: composite indexes `(user_id, event_time)` and `(product_id, event_type)`.
* Code strategy: add an `initIndexes()` method that runs after `createTable()`.

---

**Q9. What materialized views or summary tables would accelerate common business intelligence queries?**
**A (Strategy):**

* Since BI queries will hit 285M rows, create summary layers:

  * `daily_revenue` (date, revenue, unique\_customers).
  * `product_perf` (product\_id, views, carts, purchases, conversion\_rate).
  * `customer_rfm` (user\_id, recency, frequency, monetary).
* Refresh nightly with `REFRESH MATERIALIZED VIEW CONCURRENTLY`.
* Code strategy: add SQL scripts to `answerQueryAnswer()` for generating these summaries.

---






## **Task 3 – Customer Engagement & Lifetime Analysis (Deep Dive: Q1–Q3)**

---

### **Q1. Calculate Customer Lifetime Value (CLV) and Ranking within Segments**

CLV represents the **total monetary value a customer generates** over their lifetime. Ranking within segments (e.g., `category_l1`) identifies the **highest-value customers per category**.

**SQL & Analysis Strategy:**

* **Step 1:** Aggregate purchases per customer and segment:

  * Compute `SUM(price)` grouped by `user_id` and optionally `category_l1`.
  * Ensures one row per customer (or customer-category pair).

* **Step 2:** Rank customers within segments:

  * Use `RANK()` or `DENSE_RANK()` window function.
  * Partition by `category_l1` and order by CLV descending.

* **Step 3 (Optional):** Identify top-tier customers:

  * Filter by rank ≤ 10 or a percentile cutoff (e.g., top 20%).

**Business Implication:**

* High-ranking customers → target for **premium offers or loyalty programs**.
* Low CLV customers → opportunities to **increase engagement** or reduce churn.

**Key SQL Functions / Concepts:**

* `SUM(price)` (aggregation)
* `GROUP BY user_id [, category_l1]`
* `RANK() OVER (PARTITION BY category_l1 ORDER BY SUM(price) DESC)` (window function)

---

### **Q2. Identify Customers with Longest Gaps Between Purchase Events**

Long gaps in purchase behavior indicate **low engagement or churn risk**. Measuring gaps allows **retention-focused interventions**.

**SQL & Analysis Strategy:**

* **Step 1:** Filter only purchase events:

  * `event_type = 'purchase'` or `has_purchase = TRUE`.

* **Step 2:** Compute consecutive purchase gaps:

  * Use `LAG(event_time) OVER (PARTITION BY user_id ORDER BY event_time)` to get previous purchase.
  * Calculate gap: `event_time - previous_event_time`.

* **Step 3:** Aggregate per user:

  * Compute `MAX(gap)` per `user_id` to find the **longest gap**.

* **Step 4 (Optional):** Rank customers by longest gap:

  * Identify users at **highest churn risk**.

**Business Implication:**

* Customers with **long gaps** → candidates for **re-engagement campaigns**.
* Short gaps → highly engaged users; monitor for cross-sell opportunities.

**Key SQL Functions / Concepts:**

* `LAG()` (window function for previous value)
* `MAX()` (aggregation)
* `PARTITION BY user_id ORDER BY event_time`
* Timestamp arithmetic (`event_time - previous_event_time`)

---

### **Q3. Rolling 30-Day Active Users & Month-over-Month Retention**

Rolling metrics capture **trends in user engagement**, while MoM retention evaluates **returning user behavior**.

**SQL & Analysis Strategy:**

**Step A – Rolling 30-Day Active Users**

* **Step 1:** Aggregate daily active users:

  * `COUNT(DISTINCT user_id)` grouped by `DATE(event_time)`.

* **Step 2:** Compute rolling 30-day active users:

  * Use **window function**:

    * `SUM(daily_users) OVER (ORDER BY day ROWS BETWEEN 29 PRECEDING AND CURRENT ROW)`
  * Ensures **current day + previous 29 days** are included.

* **Step 3 (Optional):** Fill missing days if needed:

  * `generate_series()` to ensure **continuous daily timeline**.

**Step B – Month-over-Month (MoM) Retention**

* **Step 1:** Bucket users by month: `DATE_TRUNC('month', event_time)`.
* **Step 2:** Identify users active in current and previous month.
* **Step 3:** Calculate retention rate:

$$
Retention_{MoM} = \frac{COUNT(users_{current} \cap users_{previous})}{COUNT(users_{previous})}
$$

**Business Implication:**

* Rolling 30-day users → monitor **trends and spikes in engagement**.
* MoM retention → evaluate **loyalty, retention programs**, and growth opportunities.

**Key SQL Functions / Concepts:**

* `COUNT(DISTINCT user_id)` (unique users)
* `DATE(event_time)` / `DATE_TRUNC('month', event_time)` (time bucketing)
* Window functions: `SUM(...) OVER (...)` for rolling metrics
* Self-joins or CTEs for retention calculations

---


In [ ]:
# -- Q1:
# -- WITH customer_clv AS (
# --     SELECT
# --         user_id,
# --         category_l1,
# --         SUM(price) AS clv
# --     FROM ecommerce_events
# --     GROUP BY user_id, category_l1
# -- )

# -- SELECT
# --     user_id,
# --     category_l1,
# --     clv,
# --     RANK() OVER (PARTITION BY category_l1 ORDER BY clv DESC) AS rank_within_category
# -- FROM customer_clv
# -- ORDER BY category_l1, rank_within_category
# -- LIMIT 500;

# -- Q2:
# -- WITH customer_lag AS (
# -- 	select
# -- 	user_id,
# -- 	event_time,
# -- 	LAG(event_time) OVER (
# -- 	  PARTITION BY user_id
# -- 	  ORDER BY event_time) AS previous_event_time
# -- 	from ecommerce_events
# -- 	where event_type = 'purchase'
# -- 	order by event_time asc
# -- )

# -- select user_id, 
# -- max(event_time - previous_event_time) AS gap
# -- from customer_lag where previous_event_time IS NOT NULL
# -- group by user_id
# -- order by gap desc

# -- Q3:
# -- WITH customer_rolling AS(
# -- SELECT DATE(event_time) AS day,
# -- COUNT(DISTINCT user_id) AS daily_users
# -- from ecommerce_events
# -- group by day
# -- )

# -- select
# -- day,
# -- SUM(daily_users) OVER (
# --         ORDER BY day
# --         ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
# --     ) AS rolling_30d_active_users
# -- from customer_rolling


## **Task 2.2 – Product Performance & Cross-Selling (Q4–Q6)**

---

### **Q4: Products with Highest Conversion Rate (View → Purchase)**

**Objective:**
Identify products that **most efficiently convert views into purchases**, i.e., which products have the highest probability that a view results in a purchase.

**Analytical Strategy:**

* **Step 1 – Count views and purchases per product:**

  * Count events where `event_type = 'view'` → denominator.
  * Count events where `event_type = 'purchase'` → numerator.

* **Step 2 – Calculate conversion rate:**

$$
\text{Conversion Rate} = \frac{\text{Number of Purchases}}{\text{Number of Views}}
$$

* **Step 3 – Handle edge cases:**

  * Avoid division by zero (products with zero views).

* **Step 4 – Rank products by conversion rate:**

  * Helps identify **highly effective products** for marketing and inventory decisions.

**Business Implication:**

* High-conversion products → good candidates for promotion, upselling, or bundling.
* Low-conversion products → may require better product detail pages or targeted campaigns.

---

### **Q5: Product Category Performance Hierarchies Using CTEs**

**Objective:**
Analyze **product performance across category hierarchies** (product → category_l3 → category_l2 → category_l1) for revenue and conversion insights.

**Analytical Strategy:**

* **Step 1 – Aggregate metrics per product:**

  * Count views, count purchases, sum revenue, compute conversion rate.

* **Step 2 – Aggregate metrics per category level:**

  * Repeat the same aggregation for `category_l3`, `category_l2`, and `category_l1`.
  * Use **CTEs** to organize calculations for each level.

* **Step 3 – Optional:**

  * Query any level as needed (product, category_l3, etc.) using the precomputed CTEs.

**Business Implication:**

* Understand **which categories and products drive revenue and conversion**.
* Helps with **inventory planning, promotion targeting, and category-level KPIs**.

---

### **Q6: Product Affinity Scores & Cross-Selling Opportunities**

**Objective:**
Identify **products frequently purchased together** to inform cross-selling, bundling, and recommendation strategies.

**Analytical Strategy:**

* **Step 1 – Identify multi-product purchase sessions:**

  * Aggregate `product_id`s per `user_session`.
  * Filter sessions with more than 1 purchase.

* **Step 2 – Unnest products into rows:**

  * Convert arrays of products into separate rows per session.

* **Step 3 – Generate product pairs per session:**

  * Self-join on `user_session` to get all product combinations.
  * Keep unique pairs (`product1 < product2`) to avoid duplicates.

* **Step 4 – Aggregate pair counts across sessions:**

  * Count sessions containing both products → numerator.
  * Count sessions containing `product1` → denominator.

* **Step 5 – Compute affinity score:**

$$
\text{Affinity Score} = \frac{\text{Sessions with both products}}{\text{Sessions with product1}}
$$

* **Step 6 – Interpret results:**

  * Score ≈ 1 → products always bought together → strong cross-sell.
  * Score ≈ 0 → products rarely bought together → weak cross-sell.

**Business Implication:**

* High-affinity pairs → promote as bundles or recommendations.
* Low-affinity pairs → explore new cross-sell opportunities or ignore.

---



In [ ]:
# -- Q1
# -- SELECT product_id,
# -- round((COUNT(event_type) FILTER (WHERE event_type = 'purchase')* 1.0/
# -- NULLIF(COUNT(event_type) FILTER (WHERE event_type = 'view'), 0)), 5) as conversion_rate
# -- FROM ecommerce_events
# -- group by product_id



# -- Q2
# -- WITH product_level_stats AS(
# -- 	SELECT
# -- 	product_id,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'view') as views_count,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'purchase') as purchases_count,
# -- 	SUM(price) as reveneue,
# -- 	round((COUNT(event_type) FILTER (WHERE event_type = 'purchase')* 1.0/
# -- 	NULLIF(COUNT(event_type) FILTER (WHERE event_type = 'view'), 0)), 5) as conversion_rate
# -- 	FROM ecommerce_events
# -- 	group by product_id
# -- 	),
# -- category_l3_stats AS(
# -- 	SELECT
# -- 	category_l3,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'view') as views_count,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'purchase') as purchases_count,
# -- 	SUM(price) as reveneue,
# -- 	round((COUNT(event_type) FILTER (WHERE event_type = 'purchase')* 1.0/
# -- 	NULLIF(COUNT(event_type) FILTER (WHERE event_type = 'view'), 0)), 5) as conversion_rate
# -- 	FROM ecommerce_events
# -- 	group by category_l3
# -- 	),
# -- category_l2_stats AS(
# -- 	SELECT
# -- 	category_l2,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'view') as views_count,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'purchase') as purchases_count,
# -- 	SUM(price) as reveneue,
# -- 	round((COUNT(event_type) FILTER (WHERE event_type = 'purchase')* 1.0/
# -- 	NULLIF(COUNT(event_type) FILTER (WHERE event_type = 'view'), 0)), 5) as conversion_rate
# -- 	FROM ecommerce_events
# -- 	group by category_l2
# -- 	),
# -- category_l1_stats AS(
# -- 	SELECT
# -- 	category_l1,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'view') as views_count,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'purchase') as purchases_count,
# -- 	SUM(price) as reveneue,
# -- 	round((COUNT(event_type) FILTER (WHERE event_type = 'purchase')* 1.0/
# -- 	NULLIF(COUNT(event_type) FILTER (WHERE event_type = 'view'), 0)), 5) as conversion_rate
# -- 	FROM ecommerce_events
# -- 	group by category_l1
# -- 	)



# -- Q3
# -- with purchase_sessions as (
# -- 	select user_session,
# -- 	ARRAY_AGG(product_id) as products_arr,
# -- 	COUNT(event_type) FILTER (WHERE event_type = 'purchase') as purchases_count
# -- 	from ecommerce_events
# -- 	group by user_session
# -- ),

# -- purchase_sessions_multi as (
# -- select user_session, UNNEST(products_arr) as product_id from purchase_sessions
# -- where purchases_count > 1
# -- ),

# -- purchase_cross as (SELECT 
# --         p1.user_session, 
# --         p1.product_id AS product1, 
# --         p2.product_id AS product2
# -- from purchase_sessions_multi p1 join purchase_sessions_multi p2
# -- on p1.user_session = p2.user_session where p1.product_id < p2.product_id
# -- ),

# -- pair_counts AS (
# --     SELECT 
# --         product1, 
# --         product2, 
# --         COUNT(DISTINCT user_session) AS pair_sessions
# --     FROM purchase_cross
# --     GROUP BY product1, product2
# -- ),

# -- product_sessions AS (
# --     SELECT 
# --         product_id, 
# --         COUNT(DISTINCT user_session) AS product_sessions
# --     FROM purchase_sessions_multi
# --     GROUP BY product_id
# -- )
# -- SELECT 
# --     pc.product1,
# --     pc.product2,
# --     pc.pair_sessions,
# --     ps.product_sessions,
# --     ROUND(pc.pair_sessions::numeric / ps.product_sessions, 5) AS affinity_score
# -- FROM pair_counts pc
# -- JOIN product_sessions ps
# --   ON pc.product1 = ps.product_id
# -- ORDER BY affinity_score DESC
# -- LIMIT 50;



---

## **Q7: Identify Seasonal Trends & YoY Growth** *(not applicable for 1-month data, but structured approach if multi-year data were available)*

**Objective:**
Identify seasonal patterns in purchases or views and compute **year-over-year growth**.

**Analytical Strategy:**

* **Step 1 – Aggregate data by period:**

  * Aggregate `event_type = 'purchase'` by `MONTH(event_time)` and `YEAR(event_time)` (or by week/day for intra-month trends).
  * Compute counts or revenue per period.

* **Step 2 – Compute growth rates:**
  $$
  \text{YoY Growth} = \frac{\text{Current Period Value} - \text{Value in Same Period Last Year}}{\text{Value in Same Period Last Year}}
  $$

  * For seasonal trends, plot counts by month/week to see spikes.

* **Step 3 – Handle edge cases:**

  * Missing data for some months → treat as 0 or NULL.
  * Consider outlier days (e.g., Black Friday) separately to avoid skewing averages.

**Business Interpretation:**

* Helps marketing plan **campaigns aligned with seasonal spikes**.
* Inventory planning: stock more during high-season months.
* Detect long-term growth or decline trends.

**SQL Structure Guidance:**

* Use a **CTE** to aggregate counts/revenue per period.
* Use `LAG()` or joins to calculate YoY growth.
* Optional: `ROW_NUMBER()` or `RANK()` to detect top seasonal months.

---

## **Q8: Peak Shopping Hours & Day-of-Week Patterns**

**Objective:**
Understand **when users engage most** (hour of day or weekday), both in terms of volume and efficiency.

**Analytical Strategy:**

* **Step 1 – Aggregate events by hour/day:**

  * Use `sessionStartHour` and `sessionDayOfWeek`.
  * Count `views` and `purchases`.

* **Step 2 – Calculate distribution and conversion rate:**
  $$
  \text{Conversion Rate} = \frac{\text{Purchases in Hour/Day}}{\text{Views in Hour/Day}}
  $$

  * Compute share of total purchases and views per hour/day.

* **Step 3 – Handle edge cases:**

  * Missing hours/days → fill zeros or include all hours 0–23 and weekdays Mon–Sun.
  * Adjust for partial sessions at midnight.

**Business Interpretation:**

* Timing of **push notifications or campaigns**.
* Allocate support or logistics resources during peak hours.
* Detect unusual spikes in off-peak hours (possible fraud or bots).

**SQL Structure Guidance:**

* **CTEs** to aggregate purchases and views separately.
* **Window functions** to calculate shares and ranks.
* **Join** views and purchases to compute conversion rate per hour/day.

---

## **Q9: Moving Averages & Anomaly Detection**

**Objective:**
Smooth daily purchase/revenue trends and **identify anomalies** (unexpected spikes or drops).

**Analytical Strategy:**

* **Step 1 – Aggregate daily metrics:**

  * Count `session_count` or sum `daily_revenue` grouped by `DATE(event_time)`.

* **Step 2 – Compute moving average:**
  $$
  \text{MA}*n = \frac{\text{Day}*{t-n+1} + \ldots + \text{Day}_t}{n}
  $$

  * Use a 3–7 day rolling window depending on dataset length.

* **Step 3 – Identify anomalies:**
  $$
  \text{Deviation} = \frac{\text{Actual} - \text{MA}}{\text{MA}}
  $$

  * Flag days where |Deviation| > threshold (e.g., 30%).
  * Optionally detect both spikes and drops.

* **Step 4 – Handle edge cases:**

  * First `n-1` days → partial window.
  * Low volume days → may appear as anomalies naturally.
  * Consider separating out special promo/holiday days.

**Business Interpretation:**

* Alerts marketing, operations, and finance teams to unusual trends.
* Supports **forecasting and capacity planning**.
* Helps detect fraud or technical issues if spikes/drops are unexplained.

**SQL Structure Guidance:**

* **CTE 1:** Aggregate daily counts and revenue.
* **CTE 2:** Compute moving average using `AVG() OVER (ORDER BY event_date ROWS BETWEEN n-1 PRECEDING AND CURRENT ROW)`.
* **Final SELECT:** Compute deviation and flag anomalies using `CASE WHEN ABS(...) > threshold THEN TRUE ELSE FALSE END`.

---



In [ ]:
# Q1
# unapplicable

# Q2
# WITH sessions_per_hour_purchase AS (
# 	SELECT
# 		sessionStartHour,
# 		COUNT(user_session) as session_count
# 	FROM
# 		ecommerce_events
# 	WHERE
# 		event_type = 'purchase'
# 	GROUP BY
# 		sessionStartHour
# ),

# sessions_per_hour_view AS (
# 	SELECT
# 		sessionStartHour,
# 		COUNT(user_session) as session_count
# 	FROM
# 		ecommerce_events
# 	WHERE
# 		event_type = 'view'
# 	GROUP BY
# 		sessionStartHour
# )

# SELECT
# 	p.sessionStartHour,
# 	p.session_count * 1.0 / SUM(p.session_count)
# 	over () AS purchase_share,
# 	v.session_count * 1.0 / SUM(v.session_count)
# 	over () AS view_share,
# 	p.session_count * 1.0 / v.session_count as conversion_rate_per_day
# FROM
# 	sessions_per_hour_purchase p
# FULL JOIN
# 	sessions_per_hour_view v
# ON
# 	p.sessionStartHour = v.sessionStartHour
# ORDER BY
# 	sessionStartHour asc


# WITH sessions_per_day_purchase AS (
# 	SELECT
# 		sessionDayOfWeek,
# 		COUNT(user_session) as session_count
# 	FROM
# 		ecommerce_events
# 	WHERE
# 		event_type = 'purchase'
# 	GROUP BY
# 		sessionDayOfWeek
# ),

# sessions_per_day_views AS (
# 	SELECT
# 		sessionDayOfWeek,
# 		COUNT(user_session) as session_count
# 	FROM
# 		ecommerce_events
# 	WHERE
# 		event_type = 'view'
# 	GROUP BY
# 		sessionDayOfWeek
# )

# SELECT
# 	p.sessionDayOfWeek,
# 	p.session_count * 1.0 / SUM(p.session_count)
# 	over () AS purchase_share,
# 	v.session_count * 1.0 / SUM(v.session_count)
# 	over () AS view_share,
# 	p.session_count * 1.0 / v.session_count as conversion_rate_per_day
# FROM
# 	sessions_per_day_purchase p
# FULL JOIN 
# 	sessions_per_day_views v
# ON
# 	p.sessionDayOfWeek = v.sessionDayOfWeek
# ORDER BY
# 	conversion_rate_per_day desc

# Q3
# WITH daily_count AS (
#     SELECT
#         DATE(event_time) AS event_date,
#         COUNT(user_session) AS session_count,
#         SUM(price) AS daily_revenue
#     FROM
#         ecommerce_events
#     WHERE
#         event_type = 'purchase'
#     GROUP BY event_date
# ),
# moving_average AS (
#     SELECT
#         event_date,
#         session_count,
#         daily_revenue,
#         AVG(session_count) OVER (ORDER BY event_date
#             ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS moving_avg_session,
#         AVG(daily_revenue) OVER (ORDER BY event_date
#             ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS moving_avg_revenue
#     FROM
#         daily_count
# )
# SELECT *,
#     CASE 
#         WHEN ABS((session_count - moving_avg_session) / moving_avg_session) > 0.30
#         THEN TRUE ELSE FALSE 
#     END AS is_anomaly_session,
#     CASE 
#         WHEN ABS((daily_revenue - moving_avg_revenue) / moving_avg_revenue) > 0.30
#         THEN TRUE ELSE FALSE 
#     END AS is_anomaly_revenue
# FROM 
#     moving_average;



---

## **Q1 – Implementing RFM Analysis**

**Objective:**
Compute **Recency, Frequency, and Monetary** metrics per customer to quantify engagement and value.

**Analytical Strategy:**

1. **Recency:**

   * Days since last purchase: `CURRENT_DATE - MAX(event_time)` per customer.
2. **Frequency:**

   * Count of purchases per customer.
3. **Monetary:**

   * Total spend per customer.
4. **Use of SQL window functions:**

   * Can calculate percentiles (later used for segmentation).
   * Aggregate metrics per customer using `GROUP BY`.

**Solution Measures:**

* Created a **materialized view** `customer_rfm_summary` for efficiency on 65M records.
* Used **`FILTER` clauses** to count only purchases.
* Handled edge cases like customers with zero purchases or null monetary values.

**Business Interpretation:**

* Provides a **baseline for customer value**.
* Key input for segmentation, marketing campaigns, and retention strategies.

---

## **Q2 – Dynamic Customer Segmentation Based on Percentiles**

**Objective:**
Classify customers into segments (Power Users, Quick Buyers, Browsers/Bots, Disengaged) based on **purchase behavior**.

**Analytical Strategy:**

1. Compute **percentiles** (or numeric thresholds) for RFM metrics.
2. **Define segments using `CASE WHEN`:**

   * Power Users → recent, frequent, high spend
   * Quick Buyers → recent, moderate frequency, moderate spend
   * Browsers/Bots → low frequency & low spend
   * Disengaged → remaining customers
3. **Iterative adjustment:**

   * Initially used percentile thresholds → didn’t work due to skewed data (many zeros).
   * Switched to **numeric thresholds** derived from actual RFM distributions.
4. **Order of `CASE WHEN` matters:**

   * Ensure highest-value segments are matched first.

**Solution Measures:**

* Used **numeric thresholds**: e.g., `frequency > 5 AND monetary >= 500` for Power Users.
* Validated segment distribution to ensure **all four segments exist**.

**Business Interpretation:**

* Identifies **high-value vs low-value customers**.
* Useful for marketing, loyalty programs, promotions, and targeted campaigns.

---

## **Q3 – Customer Churn Probability Using Cohort Analysis**

**Objective:**
Estimate the probability that a customer stops purchasing over time using cohort analysis.

**Analytical Strategy:**

1. Define **cohorts** based on first purchase date.
2. Track **retention over subsequent periods**.
3. Calculate churn probability:

[
\text{Churn Probability} = 1 - \text{Retention Rate}
]

**Measures Taken:**

* With only **one month of data**, true churn cannot be calculated.
* Marked this analysis as **unapplicable**.
* Optional alternative: compute **short-term engagement proxies** (e.g., single vs multiple purchases in the month).

**Business Interpretation:**

* True churn probability requires multi-period data.
* Stakeholders would use it for retention campaigns and revenue forecasting once enough historical data is available.

---



In [ ]:
# Q2
# WITH customer_rfm_with_percentiles as (
# SELECT *,
#        PERCENT_RANK() OVER (ORDER BY recency ASC) AS recency_percentile,
#        PERCENT_RANK() OVER (ORDER BY frequency DESC) AS frequency_percentile,
#        PERCENT_RANK() OVER (ORDER BY monetary DESC) AS monetary_percentile
# FROM customer_rfm
# ),
# customer_segments AS (
# SELECT *,
#     CASE
#         WHEN frequency > 5
#              AND monetary >= 500 THEN 'Power Users'
#         WHEN frequency BETWEEN 1 AND 5
#              AND monetary >= 100 THEN 'Quick Buyers'
#         WHEN frequency = 0
#              OR monetary < 100 THEN 'Browsers/Bots'
#         ELSE 'Disengaged Users'
#     END AS customer_segment
# FROM customer_rfm_summary
# )
# SELECT customer_segment,
#        COUNT(*) AS num_users
# FROM customer_segments
# GROUP BY customer_segment
# ORDER BY num_users DESC;


---

## **Task 3 – Advanced Aggregations & Reporting (Q7–Q9)**

---

### **Q7: Using PIVOT Operations for Category Performance by Month**

**Objective:**
Transform detailed event-level data into a **cross-tab format**, where each row is a category and each column represents a month, showing a metric like revenue or purchase count.

**Analytical Strategy:**

1. **Step 1 – Identify the grouping and measure:**

   * Group by `category_l1` (or `category_l2/l3`).
   * Metric: total purchases, revenue, or session count.
2. **Step 2 – Check time granularity:**

   * Extract month from `event_time`.
   * Must have **multiple months** for pivot to be meaningful.
3. **Step 3 – Use PIVOT / conditional aggregation:**

   * If the SQL dialect supports `PIVOT`, use it.
   * Otherwise, use `CASE WHEN MONTH(event_time) = X THEN metric END` with `SUM()`.
4. **Step 4 – Edge cases:**

   * Categories with zero activity in a month → fill with 0 or NULL.

**Business Interpretation:**

* Quickly see which categories perform best per month.
* Identify seasonal trends and allocate inventory or marketing resources.

**Applicability in our project:**

* **Currently inapplicable** due to having only **one month of data**.

---

### **Q8: Funnel Conversion Rates (View → Cart → Purchase)**

**Objective:**
Calculate the **drop-off between funnel stages** for different segments (e.g., category, RFM group, traffic source).

**Analytical Strategy:**

1. **Step 1 – Aggregate event counts per stage:**

   * Count `view`, `cart`, and `purchase` events per segment (`category_l1`, `brand`, or customer segment).
2. **Step 2 – Calculate conversion rates:**

   * View → Cart: `cart_count / view_count`
   * Cart → Purchase: `purchase_count / cart_count`
   * Full funnel: `purchase_count / view_count`
   * Use `NULLIF()` to avoid division by zero.
3. **Step 3 – Optional segmentation:**

   * By category (`category_l1`) → shows product-level impact.
   * By customer type (RFM segments) → shows behavioral differences.
4. **Step 4 – Business interpretation:**

   * Identify bottlenecks in the funnel.
   * Prioritize interventions for high-impact segments.

**SQL Implementation:**

* We created a CTE `category_seg` that counts events per stage.
* Calculated `vc_conversion_rate`, `cp_conversion_rate`, and `vp_conversion_rate`.

---

### **Q9: Statistical Functions for Product Price Analysis**

**Objective:**
Analyze **price distributions** and **relationships** with engagement metrics to inform pricing and merchandising decisions.

**Analytical Strategy:**

1. **Step 1 – Remove extreme/luxury outliers:**

   * Use `isExtremeOutlier = false` and `isLuxuryItem = false`.
2. **Step 2 – Grouping:**

   * By `category_l1` or `(category_l1, brand)` to capture meaningful subgroups.
3. **Step 3 – Apply statistical functions:**

   * Average price: `AVG(price)`
   * Price variability: `STDDEV(price)`
   * Correlation with engagement metrics:

     * `CORR(price, popularityScore)` → effect of price on product popularity
     * Avoid `CORR(price, categoryConversionRate)` if variance is too low
   * Use `FILTER (WHERE metric IS NOT NULL)` to prevent null issues.
4. **Step 4 – Business interpretation:**

   * Categories with high stddev → mixed pricing, may need segmentation.
   * Strong correlation → pricing affects conversion/popularity.
   * Weak correlation → other factors drive engagement; pricing less sensitive.

**SQL Implementation:**

* Aggregated by category and brand, filtered out outliers.
* Applied `AVG`, `STDDEV`, and `CORR` with proper null handling.

---


In [ ]:
# Q1
# inaplicable

# Q2
# WITH category_seg AS (
#     SELECT
#         category_l1,
#         SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS purchase_count,
#         SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) AS view_count,
#         SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) AS cart_count
#     FROM 
#         ecommerce_events
#     GROUP BY
#         category_l1
# )
# SELECT
#     category_l1,
#     cart_count * 1.0 / NULLIF(view_count, 0)   AS vc_conversion_rate, 
#     purchase_count * 1.0 / NULLIF(cart_count, 0) AS cp_conversion_rate, 
#     purchase_count * 1.0 / NULLIF(view_count, 0) AS vp_conversion_rate 
# FROM
#     category_seg
# ORDER BY
#     vp_conversion_rate DESC;

# Q3
# SELECT
#     category_l1,
#     brand,
#     AVG(price) AS price_average,
#     STDDEV(price) AS price_std,
#     CORR(price, popularityScore) 
#         FILTER (WHERE popularityScore IS NOT NULL) AS corr_price_vs_popularity_score
# FROM
#     ecommerce_events
# WHERE
#     isExtremeOutlier = false 
#     AND isLuxuryItem = false
# GROUP BY 
#     category_l1, brand
# ORDER BY
#     category_l1, brand;


